# Bulk Download GIS Data from National Levee Database

This notebook demonstrates how to programmatically "bulk download" core GIS data from the [National Levee Database](https://levees.sec.usace.army.mil/#/)(NLD) using [GDAL/OGR Python APIs](https://gdal.org/python/). NLD GIS data is available for public access via both ESRI ArcGIS services and OGC services. In this notebook, we focus on retrieving data from the ArcGIS [Feature Service](https://developers.arcgis.com/rest/services-reference/enterprise/feature-service.htm) REST APIs. 

The offical "User Guide - NLD Web-GIS services (pdf)" (shown in the snapshot below) provides instructions on data access using ArcMap, ArcGIS Pro and QGIS. In addition, [ArcGIS Online Map Viewer](https://www.arcgis.com/home/webmap/viewer.html) is also a good tool for quick visualization. 

<img src="statics/images/NLD_gis.png" alt="drawing" width="800"/>

## Available Layers

Here we import some libs we will be using. 

In [1]:
import os
import requests
import pandas as pd
from werkzeug.utils import secure_filename
from osgeo import ogr

As the NLD FeatureService endpoint shown in the snapshot above, we here append "/?f=pjson" to the url to retrieve metadata for all layers served under the "NLD2_PUBLIC" group. We use "requests" to make a HTTP GET request to the endpoint and extract the "layers" key from the returned JSON data. We put the data in a dataframe.

In [2]:
url_feature_server = "https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer/"
layer_def = requests.get(url_feature_server + '/?f=pjson').json()

df = pd.DataFrame(layer_def["layers"])
df

,id,name,parentLayerId,defaultVisibility,subLayerIds,minScale,maxScale,type,geometryType
0,0,Boreholes,-1,False,None,0,0,Feature Layer,esriGeometryPoint
1,1,Crossings,-1,False,None,0,0,Feature Layer,esriGeometryPoint
2,2,Levee Stations,-1,False,None,0,0,Feature Layer,esriGeometryPoint
3,3,Piezometers,-1,False,None,0,0,Feature Layer,esriGeometryPoint
4,4,Pump Stations,-1,False,None,0,0,Feature Layer,esriGeometryPoint
5,5,Relief Wells,-1,False,None,0,0,Feature Layer,esriGeometryPoint
6,6,Alignment Lines,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
7,7,Closure Structures,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
8,8,Cross Sections,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
9,9,Embankments,-1,True,None,0,0,Feature Layer,esriGeometryPolyline


## Bulk Download all Features from a Single Layer

Here we write up a reusable function that downloads a single layer data. Later we will appy the this function to all layers. The library we use for interacting with the ArcGIS FeatureService is [GDAL/OGR Python APIs](https://gdal.org/python/). Since a Feature Service layer could contain a lot of features, the total number may be far beyond how many features a single REST API call could return. It is necessary to split a "big call" into multiple "smaller calls" to retrieve data in chunks and piece them up in the end. There are different ways to do it. The tip of using GDAL/OGR is we should explicitly add a "resultRecordCount=XXXXX" to activate the pagination. To determine the best value for "resultRecordCount" some "trial-and-errors" might be needed. we found 1,000 works well in this case. Note that even the ArcGIS server has a relative large cap, it is still recommended to explicitly set a smaller 'resultRecordCount" as a 'big call' may cause unexpected errors, such as time-out or wrong query result in some extreme cases.

In [3]:
# create a local folder where the downloaded data will be saved
download_folder = os.path.abspath("./download")
!mkdir -p {download_folder}

Below is the function that downloads a single FeatureLayer to local shapefiles

In [4]:
def featurelayer2shapefile_ogr(featurelayer_url, shapefile_path):
    """
    Save all data of a ArcGIS FeatureService layer to a local Shapefiles.
    This function uses GDAL/OGR Python APIs (https://gdal.org/python/)
    
    Parameters
    ---------
    featurelayer_url: string
         The REST endpioint to a specific ArcGIS FeatureService layer. 
         eg: https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer/0/
    shapefile_path: string
         The full path of shapefiles to be created locally.
         The file extension is expected to be "*.shp".
         Existing shapefiles will be overwritten.
         eg: ~/work/download/myshp.shp
    
    Returns
    ---------
    No returns
    
    """
    
    # Here we construct a Query against the FeatureService layer to return all features
    # where=1+%3D+1: (decoded: 1 = 1) a always true where clause that will hit on every feature
    # outfields=*: reture all attributes
    # f=geojson: return results in GeoJSON format
    # resultRecordCount=1000: only return 1,000 features per query and make multiple queries if needed
    # orderByFields=OBJECTID+ASC: sort results by OBJECTID
    # the above query can also be performed using the ogr2ogr command
    # ogr2ogr -overwrite -f 'ESRI Shapefile' <Shapefile> <ArcServerFeatureServer>/<LayerID>/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC
    query_url = '{}/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC'.format(featurelayer_url)
    print(query_url)
    ds = ogr.Open(query_url)
    layer = ds.GetLayerByIndex(0)
    # total number of features retrieved from the rest endpoint
    feature_count = layer.GetFeatureCount()
    print("Input Feature Count: {:,}".format(feature_count))
    # create a local shapefile
    driver_out = ogr.GetDriverByName("ESRI Shapefile")
    # remove output shapefile if it already exists
    if os.path.exists(shapefile_path):
        driver_out.DeleteDataSource(shapefile_path)
    ds_out = driver_out.CreateDataSource(shapefile_path)
    # copy GeoJSON layer to local Shapefile files
    layer_out = ds_out.CopyLayer(layer, "layer")
    print("Output Feature Count: {:,} at {}".format(layer_out.GetFeatureCount(), shapefile_path))
    # save everything
    ds = None
    ds_out = None


## Bulk Download All NLD Layers (~40 mins)

In [5]:
# a list of layers to download (by ids)
id_list = [4]
# uncomment the codes below to download all layers
#id_list = range(0, df.shape[0])

def row_func(row):
    """
    This function will be applied to each row of the above dataframe 
    For each row (layer), it calls function featurelayer2shapefile_ogr()
    to download the layer data to local shapefiles
   
    Parameters
    ---------
    row: pandas.Series
    
    Returns
    ---------
    No returns
    
    """
    name = row["name"]    
    id = row["id"]
    if id_list is not None and type(id_list) is list:
        if int(id) not in id_list:
            return
    name_safe = secure_filename(name)
    shp_path = os.path.join(download_folder, '{}.shp'.format(name_safe))
    print("-"*80)
    print("id: {}; name: {}; file: {}".format(id, name, shp_path))
    featurelayer_url = "{}/{}/".format(url_feature_server, id)
    featurelayer2shapefile_ogr(featurelayer_url, shp_path)

In [6]:
%%time
# bulk download the whole NLD normally takes 30~40 mins
_ = df.apply(row_func, axis=1)

--------------------------------------------------------------------------------
id: 4; name: Pump Stations; file: /home/jovyan/work/National_Levee_Database/download/Pump_Stations.shp
https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer//4//query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC
Input Feature Count: 2,637


Warning 6: Normalized/laundered field name: 'PUMPSTATION_ID' to 'PUMPSTATIO'
Warning 6: Normalized/laundered field name: 'LEVEE_STATION_CODE' to 'LEVEE_STAT'
Warning 6: Normalized/laundered field name: 'NUMBER_PUMPS' to 'NUMBER_PUM'
Warning 6: Normalized/laundered field name: 'PUMP_ELEVATION' to 'PUMP_ELEVA'
Warning 6: Normalized/laundered field name: 'INVERT_ELEVATION' to 'INVERT_ELE'
Warning 6: Normalized/laundered field name: 'MAX_DESIGN_HEAD' to 'MAX_DESIGN'
Warning 6: Normalized/laundered field name: 'FEATURE_NAME' to 'FEATURE_NA'
Warning 6: Normalized/laundered field name: 'INTERIOR_DRAIN' to 'INTERIOR_D'
Warning 6: Normalized/laundered field name: 'FLOOD_SOURCE' to 'FLOOD_SOUR'
Warning 6: Normalized/laundered field name: 'RIVER_BASIN' to 'RIVER_BASI'
Warning 6: Normalized/laundered field name: 'BACKUP_POWER_IND' to 'BACKUP_POW'
Warning 6: Normalized/laundered field name: 'SURVEY_DATE' to 'SURVEY_DAT'
Warning 6: Normalized/laundered field name: 'COORDINATE_METHOD' to 'COORDINATE'

Output Feature Count: 2,637 at /home/jovyan/work/National_Levee_Database/download/Pump_Stations.shp
CPU times: user 509 ms, sys: 108 ms, total: 616 ms
Wall time: 2.92 s


## Quick Inspection

In [7]:
!du {download_folder} -h

4.0K	/home/jovyan/work/National_Levee_Database/download/.ipynb_checkpoints
1.3G	/home/jovyan/work/National_Levee_Database/download


In [8]:
!ls {download_folder} -alh

total 1.3G
drwxr-xr-x 3 jovyan users 4.0K Mar 14 02:19 .
drwxr-xr-x 8 jovyan users  228 Mar 14 02:18 ..
-rw-r--r-- 1 jovyan users 111M Mar  9 22:03 Alignment_Lines.dbf
-rw-r--r-- 1 jovyan users  145 Mar  9 22:00 Alignment_Lines.prj
-rw-r--r-- 1 jovyan users  89M Mar  9 22:03 Alignment_Lines.shp
-rw-r--r-- 1 jovyan users 265K Mar  9 22:03 Alignment_Lines.shx
-rw-r--r-- 1 jovyan users  32M Mar 11 13:06 Boreholes.dbf
-rw-r--r-- 1 jovyan users  145 Mar 10 20:27 Boreholes.prj
-rw-r--r-- 1 jovyan users 1.8M Mar 11 13:06 Boreholes.shp
-rw-r--r-- 1 jovyan users  100 Mar 10 20:27 Boreholes.shx
-rw-r--r-- 1 jovyan users  12M Mar  9 22:03 Closure_Structures.dbf
-rw-r--r-- 1 jovyan users  145 Mar  9 22:03 Closure_Structures.prj
-rw-r--r-- 1 jovyan users 544K Mar  9 22:03 Closure_Structures.shp
-rw-r--r-- 1 jovyan users  27K Mar  9 22:03 Closure_Structures.shx
-rw-r--r-- 1 jovyan users  89M Mar  9 21:53 Crossings.dbf
-rw-r--r-- 1 jovyan users  145 Mar  9 21:51 Crossings.prj
-rw-r--r-- 1 jovyan user

## References

https://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson

https://gis.stackexchange.com/questions/266897/how-to-get-around-the-1000-objectids-limit-on-arcgis-server

https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html#get-wfs-layers-and-iterate-over-features

https://gdal.org/python/

https://gdal.org/drivers/vector/geojson.html#vector-geojson

https://gdal.org/drivers/vector/esrijson.html#vector-esrijson